# Indian startup ecosystem 

Your team is trying to venture into the Indian start-up ecosystem. As the data expert of the team, you are to investigate the ecosystem and propose the best course of action.
Analyze funding received by start-ups in India from 2018 to 2021.
Separate data for each year of funding will be provided.
• In these datasets, you'll find the start-ups' details, the funding amounts received, and the investors' information

# 2.0 loading packages 

In [2]:
import pyodbc
from dotenv import dotenv_values
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import f_oneway
import statistics as stat
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

2.1 load environment variables and setting the .env files 

In [3]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"

connection =pyodbc.connect(connection_string)

# 3. Expected Workflow



&#8226; business understanding

&#8226; load packages

&#8226; load dataset

&#8226; clean datasets

&#8226; test hypothesis

&#8226; answer question

&#8226; insights and conclusion



# 4. Import data

<b>load the 2020&2021 sql dataset<b>

In [4]:
# load the 2020 sql dataset
query_2020= "Select * FROM dbo.LP1_startup_funding2020"
data_2020 = pd.read_sql(query_2020, connection)

#load the 2021 sql datasets
query_2021= "Select * FROM dbo.LP1_startup_funding2021"
data_2021= pd.read_sql(query_2021, connection)

<b> load the 2018 & 2019 csv dataset

In [5]:
#load 2018 csv dataset
data_2018=pd.read_csv('startup_funding2018.csv')

#load data 2019 csv files
data_2019=pd.read_csv('startup_funding2019.csv')

# 5. Cleaning the dataset

<b> Step 1 - add and drop some columns

In [6]:
# Add a Year Column (this will help in concating and merging the data sets)
data_2018['Year']='2018'
data_2019['Year']='2019'
data_2020['Year']='2020'
data_2021['Year']='2021'

In [7]:
# drop the column we will not be using from data 2019,2020 and 2021
data_2019.drop(['Founded', 'Founders', 'Investor'], axis=1, inplace=True)
data_2020.drop(['Founded', 'Founders', 'Investor'], axis=1, inplace=True)
data_2021.drop(['Founded', 'Founders', 'Investor'], axis=1, inplace=True)

<b> Step 2 - renaming common columns into common names  

In [8]:
#renaming the 2018 dataset columns
data_2018=data_2018.rename(columns={'Company Name': 'Company_Brand', 'Round/Series': 'Stage',
                                    'Industry': 'Sector', 'Amount': 'Amount', 'About Company': 'BIO', 'Location':'HeadQuarter'})

# renaming the 2019 dataset columns
data_2019 = data_2019.rename(columns={'Company/Brand': 'Company_Brand', 'Sector': 'Sector', 'Stage': 'Stage', 'Amount($)': 'Amount',
                                      'What it does': 'BIO', 'HeadQuarter':'HeadQuarter'})

# Renaming the 2020 dataset

data_2020 = data_2020.rename(columns={'Company_Brand': 'Company_Brand', 'Sector': 'Sector', 'Amount': 'Amount',
                                       'What_it_does': 'BIO', 'Location':'HeadQuarter'})

#renaming  the  2021 dataset 
data_2021 = data_2021.rename(columns={'Company_Brand': 'Company_Brand', 'Sector': 'Sector', 'Amount': 'Amount', 
                                      'What_it_does': 'BIO', 'HeadQuarter':'HeadQuarter'})

<b> Step 3 - Harmonising the Stage column entries in 2018,2019,2020 &2021 data

In [28]:
data_2018['Stage'].value_counts()

Stage
Seed           280
Series A        73
Undisclosed     39
Angel           37
Series B        20
Other           17
Series C        16
Debt            15
Equity          13
Pre-Seed         6
Series E         5
Series D         3
Series H         1
Name: count, dtype: int64

In [10]:
#  # replacing the data 2018 unique satage value to a common value
data_2018.replace(to_replace=['Seed round', 'Seed funding', 'Early seed', 'Seed fund', 'Seed Investment', 'Seed Round', 'Seed+'], value='Seed', inplace=True)
data_2018.replace(to_replace=['Angel Round'], value='Angel', inplace=True)
data_2018.replace(to_replace=['Venture - Series Unknown'], value='Undisclosed', inplace=True)
data_2018.replace(to_replace=['Debt Financing', 'Post-IPO Debt'], value='Debt', inplace=True)
data_2018.replace(to_replace=['Private Equity', 'Post-IPO Equity'], value='Equity', inplace=True)
data_2018.replace(to_replace=['Corporate Round', 'Grant', 'Secondary Market', 'Non-equity Assistance', 'Funding Round'], value='Other', inplace=True)
data_2018.replace(to_replace=['Pre-Series B', 'Pre-series B'], value='Pre-Seed', inplace=True)

In [27]:
#remove non-standard entries (https)

data_2018= data_2018[data_2018['Stage'].str.startswith('https')==False]


In [31]:
data_2019['Stage'].value_counts()

Stage
Series A    10
Pre-Seed     9
Series B     9
Series D     4
Series C     3
Seed         3
Other        2
Series G     1
Series E     1
Series F     1
Name: count, dtype: int64

In [30]:
# replacing the data 2019 unique satage value to a common value

data_2019.replace(to_replace=['Seed funding', 'Seed fund', 'Seed round'], value='Seed', inplace=True)
data_2019.replace(to_replace=['Pre series A', 'Pre-series A'], value='Pre-Seed', inplace=True)
data_2019.replace(to_replace=['Series B+'], value='Series B', inplace=True)
data_2019.replace(to_replace=['Post series A', 'Fresh funding'], value='Other', inplace=True)

In [32]:
data_2020['Stage'].value_counts()

Stage
Seed                     173
Pre-Series               133
Series A                  97
Series B                  59
Series C                  50
Series D                  24
Debt                      18
Pre-seed                  11
Series E and Beyond       11
Bridge                     8
Angel                      4
Series H                   1
Seed Round & Series A      1
Mid series                 1
Name: count, dtype: int64

In [35]:
data_2020['Stage'].value_counts()

Stage
Seed                     173
Pre-Series               133
Series A                  97
Series B                  59
Series C                  50
Series D                  24
Debt                      18
Pre-seed                  11
Series E and Beyond       11
Bridge                     8
Angel                      4
Series H                   1
Seed Round & Series A      1
Mid series                 1
Name: count, dtype: int64

In [34]:
# replacing the data 2020 unique satage value to a common value

data_2020.replace(to_replace=['Seed Round', 'Seed round', 'Seed funding', 'Seed A', 'Seed Funding', 'Seed Investment','Pre seed Round', 'Seed funding' 'Seed A' 'Pre-seed', 'Pre seed round', 'Pre-seed Round', 'Pre-Seed'], value='Seed', inplace=True)
data_2020.replace(to_replace=['Series A1', 'Series A-1'], value='Series A', inplace=True)
data_2020.replace(to_replace=['Series B2'], value='Series B', inplace=True)
data_2020.replace(to_replace=['Pre series B', 'Pre-Series B', 'Pre series C', 'Pre-Series B' 'Pre series C','Pre series A1', 'Pre-series', 'Pre series A', 'Pre-series A', 'Pre- series A', 'Pre Series A', 'Pre-series A1', 'Pre-series C', 'Pre-series B'], value='Pre-Series', inplace=True)
data_2020.replace(to_replace=['Series E', 'Series E2', 'Series F'], value='Series E and Beyond', inplace=True)
data_2020.replace(to_replace=['Series C, D', 'Series D', 'Series D1'], value='Series D', inplace=True)
data_2020.replace(to_replace=['Bridge Round', 'Edge', 'Bridge'], value='Bridge', inplace=True)
data_2020.replace(to_replace=['Angel Round'], value='Angel', inplace=True)

In [38]:
data_2021['Stage'].value_counts()

Stage
Seed                   295
Pre-Series             173
Series A               129
Series B                50
Series C                45
Series E and Beyond     31
Debt                    27
Series D                22
Angel                    6
Bridge                   2
PE                       1
Name: count, dtype: int64

In [37]:
# replacing the data 2021 unique satage value to a common value

data_2021.replace(to_replace=['Seed Round', 'Seed round', 'Seed funding', 'Seed A', 'Seed+', 'Seed Funding', 'Seed Investment','Pre seed Round', 'Early seed', 'Seed funding' 'Seed A' 'Pre-seed', 'Pre seed round', 'Pre-seed', 'Pre-seed Round', 'Pre-Seed'], value='Seed', inplace=True)
data_2021.replace(to_replace=['Series A1', 'Seies A', 'Series A2', 'Series A+', 'Series A-1'], value='Series A', inplace=True)
data_2021.replace(to_replace=['Series B2', 'Series B3'], value='Series B', inplace=True)
data_2021.replace(to_replace=['Pre series B', 'Pre-Series B', 'Pre series C', 'Pre-Series B' 'Pre series C','Pre series A1', 'Pre-series', 'Pre series A', 'Pre-series A', 'Pre- series A', 'Pre Series A', 'Pre-series A1', 'Pre-series C', 'Pre-series B'], value='Pre-Series', inplace=True)
data_2021.replace(to_replace=['Series E', 'Series I', 'Series F1', 'Series H', 'Series G', 'Series F2', 'Series E2', 'Series F'], value='Series E and Beyond', inplace=True)
data_2021.replace(to_replace=['Series C, D', 'Series D', 'Series D1'], value='Series D', inplace=True)
data_2021.replace(to_replace=['Bridge Round', 'Edge', 'Bridge'], value='Bridge', inplace=True)
data_2021.replace(to_replace=['Angel Round', '$300000', '$1200000', '$6000000', '$1000000'], value='Angel', inplace=True)

<b> Step 3 - Harmonising the Headquater/location column entries in 2018,2019,2020&2021 data.<b>

Harmonising the 2018 headquater columns

In [42]:
data_2018['HeadQuarter']. value_counts()

HeadQuarter
Bangalore, Karnataka, India         101
Mumbai, Maharashtra, India           94
Bengaluru, Karnataka, India          55
Gurgaon, Haryana, India              52
New Delhi, Delhi, India              51
Pune, Maharashtra, India             20
Chennai, Tamil Nadu, India           19
Hyderabad, Andhra Pradesh, India     18
Delhi, Delhi, India                  16
Noida, Uttar Pradesh, India          15
Haryana, Haryana, India              11
Jaipur, Rajasthan, India              9
Ahmedabad, Gujarat, India             6
Kolkata, West Bengal, India           6
Bangalore City, Karnataka, India      5
Indore, Madhya Pradesh, India         4
India, Asia                           4
Kormangala, Karnataka, India          3
Ghaziabad, Uttar Pradesh, India       2
Kochi, Kerala, India                  2
Bhopal, Madhya Pradesh, India         2
Thane, Maharashtra, India             2
Jodhpur, Rajasthan, India             1
Powai, Assam, India                   1
Andheri, Maharashtra, India 

In [40]:
# data cleaning of headquarter in 2018

data_2018.replace(to_replace=['Bangalore City','Bangalore'], value='Bangalore', inplace=True)
data_2018.replace(to_replace=['Delhi','New Delhi'], value='Delhi', inplace=True)
data_2018.replace(to_replace=['Cochin'], value='Kochi', inplace=True)
data_2018.replace(to_replace=['Kormangala'], value='Koramangala', inplace=True)
data_2018.replace(to_replace=['Powai','Worli'], value='Mumbai', inplace=True)
data_2018.replace(to_replace=['Uttar Pradesh', 'Andhra Pradesh'], value='Andhra Pradesh', inplace=True)
data_2018.replace(to_replace=['Trivandrum', 'India'], value='Trivandrum', inplace=True)

<b>Harmonize the 2019 data headquater column

In [53]:
data_2019['HeadQuarter'].value_counts()

HeadQuarter
Bangalore        21
Mumbai           12
Delhi            11
Noida             5
Gurugram          5
Chennai           4
Pune              2
Jaipur            2
Telangana         1
Ahmedabad         1
Haryana           1
Chandigarh        1
Surat             1
Uttar pradesh     1
Hyderabad         1
Rajasthan         1
Name: count, dtype: int64

In [44]:
#2019 headquater data cleaning
data_2019.replace(to_replace=['Delhi' , 'New Delhi'], value='Delhi', inplace=True)

<b>Harmonizing the 2020 data headquater column

In [56]:
data_2020['HeadQuarter']. value_counts

<bound method IndexOpsMixin.value_counts of 0           Chennai
1         Bangalore
2              Pune
3             Delhi
4            Indore
           ...     
1050          Delhi
1051    Undisclosed
1052         Mumbai
1053          Delhi
1054        Chennai
Name: HeadQuarter, Length: 1055, dtype: object>

In [55]:
#  cleaning of HeadQuater 2020
data_2020.replace(to_replace=['Bangalore City','Bangalore'], value='Bangalore', inplace=True)
data_2020.replace(to_replace=['Delhi','New Delhi'], value='Delhi', inplace=True)
data_2020.replace(to_replace=['Ahmadabad'], value='Ahmedabad', inplace=True)
data_2020.replace(to_replace=['Kochi'], value='Cochin', inplace=True)
data_2020.replace(to_replace=['Kormangala'], value='Koramangala', inplace=True)
data_2020.replace(to_replace=['Rajastan'], value='Rajasthan', inplace=True)
data_2020.replace(to_replace=['Powai','Worli'], value='Mumbai', inplace=True)
data_2020.replace(to_replace=['Small Towns', 'Andhra Pradesh','Uttar Pradesh'], value='Andhra Pradesh', inplace=True)
data_2020.replace(to_replace=['Hyderebad'], value='Hyderabad', inplace=True)
data_2020.replace(to_replace=['Gurugram\t#REF!'], value='Gurugram', inplace=True)
data_2020.replace(to_replace=['Orissia'], value='Orissa', inplace=True)
data_2020.replace(to_replace=['Samstipur','Samastipur, Bihar','Samsitpur'], value='Samastipur', inplace=True)
data_2020.replace(to_replace=['The Nilgiris'], value='Nilgiris', inplace=True)
data_2020.replace(to_replace=['Dhindsara', 'Haryana','Dhingsara, Haryana'], value='Dhingsara', inplace=True)
data_2020.replace(to_replace=['Tirunelveli', 'Tamilnadu'], value='Tirunelveli', inplace=True)
data_2020.replace(to_replace=['Mylapore'], value='Chennai', inplace=True)
data_2020.replace(to_replace=['Rajastan'], value='Rajasthan', inplace=True)
data_2020.replace(to_replace=['Trivandrum', 'Kerala, India'], value='Trivandrum', inplace=True)

#  Renaming some of the headquarters the are not in India to outside india in data 2020
data_2020.replace(to_replace=['Frisco', 'France','Newcastle Upon Tyne, Newcastle upon Tyne, United Kingdom','Frisco Texas, United States',
                              'Irvine, California, United States','San Francisco Bay Area, West Coast, Western US','Texas, United States',
                              'California','New York, United States','San Francisco, California, United States','San Francisco, United States',
                              'San Ramon, California','Paris, Ile-de-France, France','Plano, Texas, United States','Sydney',
                              'San Francisco Bay Area, Silicon Valley, West Coast','Bangaldesh','London, England, United Kingdom',
                              'Sydney, New South Wales, Australia','Milano, Lombardia, Italy','Palmwoods, Queensland, Australia', 
                              'France''Irvine, California, United States','Newcastle Upon Tyne, Newcastle upon Tyne United Kingdom',
                              'Shanghai, China','Jiaxing, Zhejiang, China','San Franciscao','San Francisco','New York'], 
                              value = 'Outside India', inplace=True)

# filling of missing headquarters in 2020
data_2020['HeadQuarter'].fillna('Undisclosed', inplace=True)

<b>Harmonizing the 2021 headquater data colummn

In [51]:
data_2021['HeadQuarter'].unique()

array(['Bangalore', 'Mumbai', 'Gurugram', 'New Delhi', 'Hyderabad',
       'Jaipur', 'Ahmadabad', 'Chennai', None,
       'Small Towns, Andhra Pradesh', 'Goa', 'Rajsamand', 'Ranchi',
       'Faridabad, Haryana', 'Gujarat', 'Pune', 'Thane', 'Undisclosed',
       'Cochin', 'Noida', 'Chandigarh', 'Gurgaon', 'Vadodara',
       'Gurugrama', 'Kolkata', 'Ahmedabad', 'Mohali', 'Haryana', 'Indore',
       'Powai', 'Ghaziabad', 'Nagpur', 'West Bengal', 'Patna',
       'Samsitpur', 'Lucknow', 'Telangana', 'Silvassa',
       'Thiruvananthapuram', 'Faridabad', 'Roorkee', 'Ambernath',
       'Panchkula', 'Surat', 'Coimbatore', 'Andheri', 'Mangalore',
       'Telugana', 'Bhubaneswar', 'Kottayam', 'Outside_India', 'Panaji',
       'Satara', 'Orissia', 'Jodhpur', 'Santra', 'Mountain View, CA',
       'Trivandrum', 'Jharkhand', 'Kanpur', 'Bhilwara', 'Guwahati',
       'Kochi', 'The Nilgiris', 'Gandhinagar'], dtype=object)

In [50]:
#2021 headquater data cleaning
data_2021.replace(to_replace=['London', 'New York', ' Mountain View', 'CA ', 'Beijing', '' ], value='Outside_India', inplace=True)
data_2021.replace(to_replace=['Online Media\t#REF!', 'Pharmaceuticals\t#REF!', 'Food & Beverages', 
                             'Computer Games', 'Small Towns', ' None', 'None', 'Information Technology & Services'], value='Undisclosed', inplace=True)
data_2021.replace(to_replace=['Gurugram\t#REF!' ], value='Gurugrama', inplace=True)

# Step 4 - Currency Conversion for 2018,2019,2020 & 2021 dataset

<b> Currency Conversion For 2018 & 2019 csv Files

In [ ]:
# Merge the two csv files 
data2018_2019 =pd.concat([data_2018,data_2019], ignore_index= True)


# Create a new column 'currency' based on currency symbols
data2018_2019['currency']= data2018_2019['Amount'].apply(lambda x: 'INR' if '₹' else ('USD' if '$'in x else 'USD'))


# Create columns 'amount_inr' and 'amount_usd'

data2018_2019['amount_inr'] = data2018_2019.apply(lambda row: row['Amount'] if row['currency'] == 'INR' else 0, axis=1)
data2018_2019['amount_usd'] = data2018_2019.apply(lambda row: row['Amount'] if row['currency'] == 'USD' else 0, axis=1)

# Remove the currency sysmbols and the commas 

data2018_2019['amount_inr'] = data2018_2019['amount_inr'].str.replace('₹', '').str.replace('$', '').str.replace(',', '')
data2018_2019['amount_usd']= data2018_2019['amount_usd'].str.replace('₹','').str.replace('$', '').str.replace(',','')


# Replace 'Undisclosed' values with NaN
data2018_2019['amount_inr']= data2018_2019['amount_inr'].replace('Undisclosed',np.nan, regex=True)
data2018_2019['amount_usd'] = data2018_2019['amount_usd'].replace('Undisclosed', np.nan, regex=True)

# Convert the object values into numeric values for calculation

data2018_2019['amount_inr'] = pd.to_numeric(data2018_2019['amount_inr'], errors='coerce')
data2018_2019['amount_usd']= pd.to_numeric(data2018_2019['amount_usd'],errors=['coerce'])

# Define the exchange rate
exchange_rate_inr_to_usd = 1 / 83.23  # 1 USD = 83.23 INR

# Convert 'amount to USD
data2018_2019['amount_inr']= data2018_2019['amount_inr'].apply(lambda x: x *exchange_rate_inr_to_usd if x is not None else None)
data2018_2019['amount_inr'] = data2018_2019['amount_inr'].apply(lambda x: x * exchange_rate_inr_to_usd if x is not None else None

# Create a new column 'Updated_Amount' by filling missing values in 'amount_inr' with 'amount_usd'
df18_19['Updated_Amount'] = df18_19['amount_inr'].fillna(df18_19['amount_usd'])